# Reading data from SACC with Firecrown Methods

This notebook examplifies reading information for Cluster Counts from a SACC file using Firecrown.

In [1]:
import os
import pyccl as ccl
from typing import Any, Dict
import numpy as np
import sacc
from firecrown.models.cluster_abundance import ClusterAbundance
from firecrown.models.cluster_mass_rich_proxy import ClusterMassRich, ClusterMassRichBinArgument, ClusterMassRichPointArgument
from firecrown.models.cluster_redshift_spec import ClusterRedshiftSpec, ClusterRedshiftArgument, ClusterRedshiftSpecArgument
from firecrown.models.cluster_redshift import ClusterRedshiftArgument
from firecrown.models.cluster_mass import ClusterMassArgument

import matplotlib.pyplot as plt



## Define Cluster Module objects and Read SACC file

We first need to read the fits file generated by SACC with the cluster data. Then, we need to instantiate `ClusterMass` and`ClusterRedshift` objects, which contain the `gen_from_tracer` methods. In our case, we are using `ClusterMassRich` and `ClusterRedshiftSpec`.

In [ ]:
sacc_data = sacc.Sacc.load_fits("/pbs/home/e/ebarroso/firecrown/examples/cluster_number_counts/cluster_redshift_richness_sacc_data.fits")

pivot_mass = 14.625862906
pivot_redshift = 0.6

cluster_mass = ClusterMassRich(pivot_mass, pivot_redshift)
cluster_redshift = ClusterRedshiftSpec()


## Get tracer combinations from SACC

We are filtering for the survey and data that we want. We need to define which survey tracer we are interested in and the data type, which are `survey_tracer` and `cluster_counts` respectively. 

In [6]:
data_type = sacc.standard_types.cluster_counts
survey_tracer = "numcosmo_simulated_redshift_richness"
#This code will only get the tracer combinations related to the cluster counts data type.
tracers_combinations = np.array(sacc_data.get_tracer_combinations(data_type=data_type))
cluster_survey_tracers = tracers_combinations[:, 0]

#This will give as a boolean list that will say if the data belongs to the survey we are interested in.
survey_selection = cluster_survey_tracers == survey_tracer

z_tracers = np.unique(tracers_combinations[survey_selection, 1])
logM_tracers = np.unique(tracers_combinations[survey_selection, 2])

## Build Bins using the cluster module objects

Having defined the data and survey types that we want, we can now call the methods from `ClusterMassRich` and `ClusterRedshiftSpec` to construct the bins, given the tracers. The result is dictionaries whose keys are the name of the bin and the item its respectively cluster argument.

In [19]:
zbin_name_list = []
rbin_name_list = []

#the result will be a dictionary where the key are the bin names and the item is a ClusterRedshiftArgument.
z_tracer_bins: Dict[str, ClusterRedshiftArgument] = {
            z_tracer: cluster_redshift.gen_bin_from_tracer(
                sacc_data.get_tracer(z_tracer)
            )
            for z_tracer in z_tracers
        }
for z_arg in z_tracer_bins.items():
    print(f"The bin {z_arg[0]}, with bounds { z_arg[1].get_z_bounds()}")
    zbin_name_list.append(z_arg[0])
    
    
#the result will be a dictionary where the key are the bin names and the item is a ClusterMassArgument.
logM_tracer_bins: Dict[str, ClusterMassArgument] = {
            logM_tracer: cluster_mass.gen_bin_from_tracer(
                sacc_data.get_tracer(logM_tracer)
            )
            for logM_tracer in logM_tracers
        }
for r_arg in logM_tracer_bins.items():
    print(f"The bin {r_arg[0]}, with bounds {(r_arg[1].logM_obs_lower, r_arg[1].logM_obs_upper)}")
    rbin_name_list.append(r_arg[0])
    
    
    
tracer_args = [
            (z_tracer_bins[z_tracer], logM_tracer_bins[logM_tracer])
            for _, z_tracer, logM_tracer in tracers_combinations[survey_selection]
        ]

#Selecting the data correspondent to the survey
data_vector_list = list(
            sacc_data.get_mean(data_type=data_type)[survey_selection]
        )
#Selecting the indeces correspondent to the data 
sacc_indices_list = list(
            sacc_data.indices(data_type=data_type)[survey_selection]
        )

print(rbin_name_list)

print(f"The data vector is: {data_vector_list}")

#Here is how to access only one data correspondent to the given survey, the redshift bin and richness bin chosen.
tracers = (survey_tracer, zbin_name_list[0], rbin_name_list[1], )
f = sacc_data.get_data_points(tracers=tracers, data_type=data_type)

print(f"{f}")

The bin bin_z_0, with bounds (0.20009201200059548, 0.31252844367470045)
The bin bin_z_1, with bounds (0.31252844367470045, 0.42496487534880545)
The bin bin_z_2, with bounds (0.42496487534880545, 0.5374013070229103)
The bin bin_z_3, with bounds (0.5374013070229103, 0.6498377386970153)
The bin rich_0, with bounds (0.868600263885897, 1.0696106627843909)
The bin rich_1, with bounds (1.0696106627843909, 1.2706210616828848)
The bin rich_2, with bounds (1.2706210616828848, 1.4716314605813787)
The bin rich_3, with bounds (1.4716314605813787, 1.6726418594798727)
The bin rich_4, with bounds (1.6726418594798727, 1.8736522583783666)
['rich_0', 'rich_1', 'rich_2', 'rich_3', 'rich_4']
The data vector is: [389.0, 183.0, 78.0, 26.0, 4.0, 591.0, 259.0, 101.0, 25.0, 4.0, 795.0, 309.0, 96.0, 36.0, 7.0, 856.0, 304.0, 102.0, 27.0, 5.0]
[DataPoint(data_type='cluster_counts', tracers=('numcosmo_simulated_redshift_richness', 'bin_z_0', 'rich_1'), value=183, )]


## Bin Information


Here is another example on how to extract the bounds information from each Cluster Argument object.

In [21]:
for z_arg in z_tracer_bins.items():
    print(f"The bin {z_arg[0]} has the bounds {z_arg[1].get_z_bounds()}\n")
for r_arg in logM_tracer_bins.items():
    print(f"The bin {r_arg[0]} has the bounds {(r_arg[1].logM_obs_lower, r_arg[1].logM_obs_upper)}")
    print(f"The mass integration limits for the bin are {r_arg[1].get_logM_bounds()}\n")


The bin bin_z_0 has the bounds (0.20009201200059548, 0.31252844367470045)

The bin bin_z_1 has the bounds (0.31252844367470045, 0.42496487534880545)

The bin bin_z_2 has the bounds (0.42496487534880545, 0.5374013070229103)

The bin bin_z_3 has the bounds (0.5374013070229103, 0.6498377386970153)

The bin rich_0 has the bounds (0.868600263885897, 1.0696106627843909)
The mass integration limits for the bin are (13.0, 16.0)

The bin rich_1 has the bounds (1.0696106627843909, 1.2706210616828848)
The mass integration limits for the bin are (13.0, 16.0)

The bin rich_2 has the bounds (1.2706210616828848, 1.4716314605813787)
The mass integration limits for the bin are (13.0, 16.0)

The bin rich_3 has the bounds (1.4716314605813787, 1.6726418594798727)
The mass integration limits for the bin are (13.0, 16.0)

The bin rich_4 has the bounds (1.6726418594798727, 1.8736522583783666)
The mass integration limits for the bin are (13.0, 16.0)

